In [ ]:
class SensibleHeatFlux(_SurfaceFlux):
    r'''Surface turbulent sensible heat flux implemented through a bulk aerodynamic formula.

    The flux is computed from

    .. math::

        SH = r ~ c_p ~\rho ~ C_D ~ U \left( T_s - T_a \right)


    where:

    - :math:`c_p` and :math:`\rho` are the specific heat and density of air
    - :math:`C_D` is a drag coefficient (stored as ``self.Cd``, default value is 3E-3)
    - :math:`U` is the near-surface wind speed, stored as ``self.U``, default value is 5 m/s
    - :math:`r` is an optional resistance parameter (stored as ``self.resistance``, default value = 1)

    The surface temperature :math:`T_s` is taken directly from ``self.state['Ts']``,
    while the near-surface air temperature :math:`T_a` is taken as the lowest model
    level in ``self.state['Tatm']``

    Diagnostic quantity ``self.SHF`` gives the sensible heat flux in W/m2.

    Temperature tendencies associated with this flux are computed for
    ``Ts`` and for the lowest model level in ``Tatm``. All other tendencies
    (including air temperature tendencies at other levels) are set to zero.
    '''
    def __init__(self, Cd=3E-3, **kwargs):
        super(SensibleHeatFlux, self).__init__(Cd=Cd, **kwargs)
        self.add_diagnostic('SHF', 0.*self.Ts)

    def _compute_flux(self):
        # this ensure same dimensions as Ts
        #  (and use only the lowest model level)
        Ta = Field(self.Tatm[..., -1, np.newaxis], domain=self.Ts.domain)
        Ts = self.Ts
        DeltaT = Ts - Ta
        rho = self._air_density(Ta)
        #  flux from bulk formula
        self._flux = self.resistance * const.cp * rho * self.Cd * self.U * DeltaT
        self.SHF = self._flux

In [2]:
import climlab
alb = 0.25
#  State variables (Air and surface temperature)
state = climlab.column_state(num_lev=30)
#  Parent model process
rcm = climlab.TimeDependentProcess(state=state)
#  Fixed relative humidity
h2o = climlab.radiation.ManabeWaterVapor(state=state)
#  Couple water vapor to radiation
rad = climlab.radiation.RRTMG(state=state, specific_humidity=h2o.q, albedo=alb)
#  Convective adjustment
conv = climlab.convection.ConvectiveAdjustment(state=state, adj_lapse_rate=6.5)
#  Couple everything together
rcm.add_subprocess('Radiation', rad)
rcm.add_subprocess('WaterVapor', h2o)
rcm.add_subprocess('Convection', conv)


Opened data from /opt/anaconda3/envs/conda_env/lib/python3.7/site-packages/climlab/radiation/data/ozone/apeozone_cam3_5_54.nc


In [5]:
rcm.state['Tatm'].shape

(30,)

In [8]:
rcm.lev

array([ 16.66666667,  50.        ,  83.33333333, 116.66666667,
       150.        , 183.33333333, 216.66666667, 250.        ,
       283.33333333, 316.66666667, 350.        , 383.33333333,
       416.66666667, 450.        , 483.33333333, 516.66666667,
       550.        , 583.33333333, 616.66666667, 650.        ,
       683.33333333, 716.66666667, 750.        , 783.33333333,
       816.66666667, 850.        , 883.33333333, 916.66666667,
       950.        , 983.33333333])